In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
import datetime as dt
import sqlalchemy
from sqlalchemy import create_engine


import sqlite3

pd.set_option('display.max_rows', 100)

In [2]:
datetime_stmp = dt.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

db_name = f"ResidentsDatabase_{datetime_stmp}.db"

print(db_name)

ResidentsDatabase_2024-12-05_17-10-26.db


In [3]:
db_name

'ResidentsDatabase_2024-12-05_17-10-26.db'

<h2> Residents Table

In [4]:
data = pd.read_excel("main.xlsx", sheet_name="DB", parse_dates=["DateofAdmission", "DateofBirth", "RentStartDate"], dtype="O")



date_cols = ["DateofAdmission", "DateofBirth", "RentStartDate"]

float_cols = ["Rent", "Deposit"]

str_cols = [col for col in data.columns if col not in date_cols + float_cols]


    
for a_col in date_cols:
    data[a_col] = pd.to_datetime(data[a_col])
    
for a_col in float_cols:
    data[a_col] = pd.to_numeric(data[a_col],errors="raise").astype("float")
    
for a_col in str_cols:
    data[a_col] = data[a_col].astype("string")
    


data.to_csv("first_data_dump.csv")





In [5]:
data.select_dtypes(include=["string", "O"]).fillna("")

,EnrollmentID,BedID,RoomNo,Name,FathersName,ContactNumber,OtherContact,FathersContact,MothersContact,AdditionalContact,...,Course,Batch,InstituteContact,InstituteContactOther,InstituteID,GuardianName,GuardianRelation,GuardianContact1,GuardianContact2,GuardianAddress
0,2024001,1A,1,Govind Mohan Purohit,Brij Mohan Purohit,9587781827,,9929483591,,,...,RAS,B-15,,,,Brij Mohan Purohit,father,9929483591,,"Soniyana, Chittorgarh(Raj.)"
1,2024002,1B,1,Mohd. Sahil Khan,Mohmmed Ayub Khan Zai,9001203254,9950326544,9829347791,9829347791,,...,RAS,,,,,Tabis,JiJa JI,9829369458,,Manserover Jaipur
2,2024003,2A,2,Paramshiv jharwal,Shashi Kumar Meena,,,9001198085,,9261295055,...,Jee - Main,FS07,,,6927343,Shashi Kumar,father,9001198085,9694720713,
3,2024004,2B,2,shaurabh meghwal,Manilal Meghwal,8078606234,,9414758408,,,...,RPSC - AEN,,,,,,,,,
4,2024005,3A,3,Vipin Meena,Udai Lal Meena,9079952437,,9928803616,,,...,RAS,2023-24,,,,Bhavesh Pujari,Friend,8949890322,,Surya Nagar
5,2024006,3B,3,Amit Kanojiya,Santosh Kanoujiya,8369605667,8898091774,8879962782,9136288384,,...,NEET,,,,,Akash Kanojia,Mama ji,8240544912,8872759567,"Block - 74/2/5, Unit-3, Garden Reach Road, Sou..."
6,2024007,3C,3,Jalaj Lohar,Pankaj Lohar,7568763329,8949986492,8949986492,,,...,RAS,D1,,,,Anchal Lohar,Sister,9610140466,,Gopalpura
7,2024008,4A,4,Adish Singhal,Rakesh Kumar Singhal,,,9414029213,7340579213,,...,JEE,FR09,,,,Muskan Singhal,Sister,8619089705,,
8,2024009,4B,4,Tanishq Singhal,Alok Kumar Singhal,8233668109,,7976795556,,,...,NEET,TR09,,,,Muskan Singhal,Sister,8619089705,,Kumbha Marg Jaipur
9,2024010,5A,5,Saket Gautam,Kunjbihari Sharma,9829009097,,9829597419,,,...,RAS,B-13,,,,,,,,


In [6]:

# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect(f"./{db_name}")
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS residents_info')

# Create table with nullable columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS residents_info (
    EnrollmentID TEXT PRIMARY KEY NOT NULL,
    BedID TEXT NOT NULL,
    RoomNo TEXT NOT NULL,
    DateofAdmission DATE NOT NULL,                    
    Name TEXT NOT NULL,
    FathersName TEXT NOT NULL,
    DateofBirth DATE NOT NULL,
    ContactNumber TEXT ,
    OtherContact TEXT ,
    FathersContact TEXT ,
    MothersContact TEXT ,
    AdditionalContact TEXT ,
    Email TEXT,
    Address TEXT,
    BloodGroup TEXT,
    ResidentAadhar TEXT,
    FathersAadhar TEXT,
    InstituteName TEXT,
    InstituteAddress TEXT,
    Course TEXT,
    Batch TEXT,
    InstituteContact TEXT,
    InstituteContactOther TEXT,
    InstituteID TEXT,
    GuardianName TEXT,
    GuardianRelation TEXT,
    GuardianContact1 TEXT,
    GuardianContact2 TEXT,
    GuardianAddress TEXT,
    Rent REAL NOT NULL,
    Deposit REAL NOT NULL,
    RentStartDate DATE NOT NULL
        
)
''')

# Commit changes and close the connection
conn.commit()



# Insert DataFrame values into the students table
data.to_sql('residents_info', conn, if_exists='append', index=False)

# Verify by reading the data from the database
df_from_db = pd.read_sql('SELECT * FROM residents_info', conn)
# print(df_from_db)

# Close the connection
conn.close()



<h2> Electricity Table

In [7]:
data_electricity = pd.read_excel("main.xlsx", sheet_name="electricity", parse_dates=["Date"])

data_electricity.to_csv("first_electricity_data_dump.csv")

In [8]:

# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect(f"./{db_name}")
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS electricity_readings')

# Create table with nullable columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS electricity_readings (
    Date DATE PRIMARY KEY NOT NULL,
    Room_1 REAL NOT NULL,
    Room_2 REAL NOT NULL,
    Room_3 REAL NOT NULL,
    Room_4 REAL NOT NULL,
    Room_5 REAL NOT NULL,
    Room_6 REAL NOT NULL,
    Room_7 REAL NOT NULL,
    Room_8 REAL NOT NULL,
    Room_9 REAL NOT NULL,
    Room_10 REAL NOT NULL,
    Room_11 REAL NOT NULL,
    Room_101 REAL NOT NULL,
    Room_102 REAL NOT NULL,
    Room_103 REAL NOT NULL,
    Room_104 REAL NOT NULL,
    Room_105 REAL NOT NULL,
    Room_106 REAL NOT NULL,
    Room_107 REAL NOT NULL,
    Room_108 REAL NOT NULL,
    Room_109 REAL NOT NULL,
    Room_110 REAL NOT NULL,
    Room_111 REAL NOT NULL,
    Room_112 REAL NOT NULL,
    Room_113 REAL NOT NULL,
    Room_114 REAL NOT NULL,
    Room_115 REAL NOT NULL,
    Room_116 REAL NOT NULL,
    Room_117 REAL NOT NULL,
    Room_201 REAL NOT NULL,
    Room_202 REAL NOT NULL,
    Room_203 REAL NOT NULL,
    Room_204 REAL NOT NULL,
    Room_205 REAL NOT NULL,
    Room_206 REAL NOT NULL,
    Room_207 REAL NOT NULL,
    Room_208 REAL NOT NULL,
    Room_209 REAL NOT NULL,
    Room_210 REAL NOT NULL,
    Room_211 REAL NOT NULL,
    Room_212 REAL NOT NULL,
    Room_213 REAL NOT NULL,
    Room_214 REAL NOT NULL,
    Room_215 REAL NOT NULL,
    Room_216 REAL NOT NULL,
    Room_217 REAL NOT NULL,
    Meter_1_2A REAL NOT NULL,
    Meter_2_2B REAL NOT NULL,
    Meter_3_1A REAL NOT NULL,
    Meter_4_1B REAL NOT NULL,
    Meter_5_GA REAL NOT NULL,
    Meter_6_GB REAL NOT NULL,
    Meter_7_Basement REAL NOT NULL,
    Library REAL NOT NULL,
    Solar REAL NOT NULL


        
)
''')

# Commit changes and close the connection
conn.commit()



# Insert DataFrame values into the students table
data_electricity.to_sql('electricity_readings', conn, if_exists='append', index=False)

# Verify by reading the data from the database
df_from_db = pd.read_sql('SELECT * FROM electricity_readings', conn)
# print(df_from_db)

# Close the connection
conn.close()



<h3> Status Table

In [9]:
data_status =  pd.read_excel("main.xlsx", sheet_name="Status", dtype="O", parse_dates=["LastRentCalcDate", "LastElectricityCalcDate"])
data_status = data_status[["BedID", "RoomNo", "EnrollmentID", "LastRentCalcDate", "LastElectricityCalcDate", "TransDate", "RoomElectricityReading", "CumulativeElectConsumption"]]





date_cols = ["LastRentCalcDate", "LastElectricityCalcDate", "TransDate"]
float_cols = ["RoomElectricityReading", "CumulativeElectConsumption"]
str_cols = [col for col in data_status.columns if col not in date_cols + float_cols]


    
for a_col in date_cols:
    data_status[a_col] = pd.to_datetime(data_status[a_col])
    
for a_col in float_cols:
    data_status[a_col] = pd.to_numeric(data_status[a_col],errors="raise").astype("float")
    
for a_col in str_cols:
    data_status[a_col] = data_status[a_col].astype("string")
    


In [10]:
data_status


,BedID,RoomNo,EnrollmentID,LastRentCalcDate,LastElectricityCalcDate,TransDate,RoomElectricityReading,CumulativeElectConsumption
0,1A,1,2024001,2024-10-31,2024-10-31,2024-10-31,3449.5,0.0
1,1B,1,2024002,2024-10-31,2024-10-31,2024-10-31,3449.5,0.0
2,2A,2,2024003,2024-10-31,2024-10-31,2024-10-31,3940.6,0.0
3,2B,2,2024004,2024-10-31,2024-10-31,2024-10-31,3940.6,0.0
4,3A,3,2024005,2024-10-31,2024-10-31,2024-10-31,7775.3,0.0
5,3B,3,2024006,2024-10-31,2024-10-31,2024-10-31,7775.3,0.0
6,3C,3,2024007,2024-10-31,2024-10-31,2024-10-31,7775.3,0.0
7,4A,4,2024008,2024-10-31,2024-10-31,2024-10-31,2452.3,0.0
8,4B,4,2024009,2024-10-31,2024-10-31,2024-10-31,2452.3,0.0
9,5A,5,2024010,2024-10-31,2024-10-31,2024-10-31,4334.6,0.0


In [11]:
data_status.isna().sum()

BedID                         0
RoomNo                        0
EnrollmentID                  3
LastRentCalcDate              3
LastElectricityCalcDate       3
TransDate                     3
RoomElectricityReading        0
CumulativeElectConsumption    3
dtype: int64

In [12]:

# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect(f"./{db_name}")
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS status')

# Create table with nullable columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS status (
    BedID TEXT PRIMARY KEY NOT NULL,
    RoomNo TEXT NOT NULL,
    EnrollmentID TEXT UNIQUE,
    LastRentCalcDate DATE,
    LastElectricityCalcDate DATE,
    TransDate DATE,
    RoomElectricityReading REAL,
    CumulativeElectConsumption REAL
    
        
)
''')

# Commit changes and close the connection
conn.commit()



# Insert DataFrame values into the students table
data_status.to_sql('status', conn, if_exists='append', index=False)

# Verify by reading the data from the database
df_from_db = pd.read_sql('SELECT * FROM status', conn)
# print(df_from_db)

# Close the connection
conn.close()



In [13]:
df_from_db.tail(1)["EnrollmentID"].unique()

array(['2024094'], dtype=object)

<h3> Transactions Table

In [14]:
# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect(f"./{db_name}")
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS transactions')

# Create table with nullable columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    TransDate DATE NOT NULL,
    BedID TEXT NOT NULL,
    RoomNo TEXT NOT NULL,
    EnrollmentID TEXT NOT NULL,
    RoomElectricityReading REAL NOT NULL,
    TransType TEXT NOT NULL,
    PrevDueAmount REAL,
    AdditionalCharges REAL,
    Comments TEXT,
    RentThruDate DATE
        
)
''')

# Commit changes and close the connection
conn.commit()

<h3> Logs Table

In [15]:
# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect(f"./{db_name}")
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS logs')

# Create table with nullable columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS logs (
    Date DATE NOT NULL,
    BedID TEXT,
    RoomNo TEXT,
    EnrollmentID TEXT,
    Type TEXT NOT NULL,
    DB_Before TEXT,
    DB_After TEXT,
    Error TEXT,
    Comments TEXT
)
''')

# Commit changes and close the connection
conn.commit()

<h3> final Settlement

In [16]:
# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect(f"./{db_name}")
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS final_settlement')

# Create table with nullable columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS final_settlement (
    ExitDate DATE NOT NULL,
    BedID TEXT NOT NULL,
    RoomNo TEXT NOT NULL,
    Name TEXT NOT NULL,
    EnrollmentID TEXT NOT NULL,
    LastRentCalcDate DATE,
    RentThruDate DATE,
    RentDays INTEGER NOT NULL,
    Rent REAL NOT NULL,
    RentDue REAL NOT NULL,

    PrevElectricityCalcDate DATE,
    LastElectricityCalcDate DATE NOT NULL,
    PrevRoomElectricityReading REAL,
    ExitElectricityReading REAL NOT NULL,
    RoomElectricityConsumption REAL,
    CumulativeElectConsumption REAL NOT NULL,
    ElectricityCharges REAL NOT NULL,

    PrevDueAmount REAL,
    AdditionalCharges REAL,
    
    TotalAmountDue REAL NOT NULL,
    Deposit REAL NOT NULL,
    NetAmountDue REAL NOT NULL,
    Comments TEXT
    
    
    
    
    
        
)
''')

# Commit changes and close the connection
conn.commit()

<h3> Rent History

In [17]:
# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect(f"./{db_name}")
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS rent_history')

# Create table with nullable columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS rent_history (
    
    
    TransDate DATE,
    
    BedID TEXT NOT NULL,
    RoomNo TEXT NOT NULL,
    EnrollmentID TEXT,
    Name TEXT,
    
    Rent REAL,
    Deposit REAL,
    
    PrevElectricityCalcDate DATE,
    LastElectricityCalcDate DATE,
    PrevElectricityReading REAL,
    RoomElectricityReading REAL,
    RoomElectricityConsumption REAL,
    UnitsConsumed REAL,
    ElectricityCharges REAL,
    
    PrevRentCalcDate DATE,
    LastRentCalcDate DATE,
    RentDays INTEGER,
    RentDue REAL,
    TotalAmountDue REAL
        
)
''')

# Commit changes and close the connection
conn.commit()

In [18]:
# conn = sqlite3.connect(f"./{db_name}")

In [19]:
df_from_db = pd.read_sql('SELECT * FROM rent_history', conn)

In [20]:
df_from_db.columns

Index(['TransDate', 'BedID', 'RoomNo', 'EnrollmentID', 'Name', 'Rent',
       'Deposit', 'PrevElectricityCalcDate', 'LastElectricityCalcDate',
       'PrevElectricityReading', 'RoomElectricityReading',
       'RoomElectricityConsumption', 'UnitsConsumed', 'ElectricityCharges',
       'PrevRentCalcDate', 'LastRentCalcDate', 'RentDays', 'RentDue',
       'TotalAmountDue'],
      dtype='object')